https://towardsdatascience.com/yolo-object-detection-with-opencv-and-python-21e50ac599e9

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from pie_data import PIE
from os import listdir
from os.path import join

one image:

In [3]:
image = cv2.imread('D:\\skola\\ING\\LS2023\\NSIETE\\zadanie3\\images\\set01\\video_0001\\01125.png')
Width = image.shape[1]
Height = image.shape[0]
scale = 0.00392

# read class names from text file
classes = None
with open('../yolov3.txt', 'r') as f:
    classes = [line.strip() for line in f.readlines()]
print(classes)

FileNotFoundError: [Errno 2] No such file or directory: '../yolov3.txt'

In [ ]:
# generate different colors for different classes 
COLORS = np.random.uniform(0, 255, size=(len(classes), 3))

# read pre-trained model and config file
net = cv2.dnn.readNet('../yolov3.weights', '../yolov3.cfg')

# prepare the input image to run through the neural network
# create input blob 
blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False)

# set input blob for the network
net.setInput(blob)

In [ ]:
# function to get the output layer names 
# in the architecture
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    try:
        output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]
    except:
        output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

# function to draw bounding box on the detected object with class name
def draw_bounding_box(img, class_id, confidence, x, y, x_plus_w, y_plus_h):

    label = str(classes[class_id])

    color = COLORS[class_id]

    cv2.rectangle(img, (x,y), (x_plus_w,y_plus_h), color, 2)

    cv2.putText(img, label, (x-10,y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

In [ ]:
# run inference through the network
# and gather predictions from output layers
outs = net.forward(get_output_layers(net))

In [ ]:
type(outs[0][0][0])

In [ ]:
class_ids = []
confidences = []
boxes = []
conf_threshold = 0.5
nms_threshold = 0.4


for out in outs:
    for detection in out:
        scores = detection[5:]
        class_id = np.argmax(scores)
        confidence = scores[class_id]
        if confidence > 0.5:
            center_x = int(detection[0] * Width)
            center_y = int(detection[1] * Height)
            w = int(detection[2] * Width)
            h = int(detection[3] * Height)
            x = center_x - w / 2
            y = center_y - h / 2
            class_ids.append(class_id)
            confidences.append(float(confidence))
            boxes.append([x, y, w, h])

In [ ]:
indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)

for i in indices:
    try:
        box = boxes[i]
    except:
        i = i[0]
        box = boxes[i]
    
    x = box[0]
    y = box[1]
    w = box[2]
    h = box[3]
    draw_bounding_box(image, class_ids[i], confidences[i], round(x), round(y), round(x+w), round(y+h))

cv2.imshow("object detection", image)
cv2.waitKey()
    
cv2.imwrite("../outputs/yolov3/object_detection.jpg", image)
cv2.destroyAllWindows()


multiple images

In [ ]:
pie_path = '../../zadanie2/' #'/media/erik/T7/DP/PIE_data' # '../' 
pie = PIE(data_path=pie_path)
type(pie)

In [ ]:
train_list = pie._get_image_set_ids('train')
test_list = pie._get_image_set_ids('test')
val_list = pie._get_image_set_ids('val')

train_list, val_list, test_list

In [ ]:
set_list = pie._get_image_set_ids('test')
set_list

In [ ]:
set_dir = dict()
all_path = list()
all_filenames = list()

for set_ in set_list:
    set_path = join(pie._images_path, set_)
    set_dir[set_] = dict()
    
    set_content = listdir(set_path)
    for video in set_content:
        video_path = join(set_path, video)
        imgs = listdir(video_path)
        all_path += ([video_path] * len(imgs))
        all_filenames += imgs
        set_dir[set_][video] = imgs
print(set_dir)

In [ ]:
print(all_path)


In [ ]:
# initialize the list of images we'll be passing through the network
images = []
# loop over the input images (excluding the first one since we
# already classified it), pre-process each image, and update the
# `images` list
for set_key in set_dir:
	for video in set_dir[set_key]:
		for frame in set_dir[set_key][video]:
			path = '../../zadanie2/images/' + set_key + '/' + video + '/' + frame
			#print(path)
			image = cv2.imread(path)
			image = cv2.resize(image, (400, 200))
			images.append(image)

In [ ]:
plt.imshow(images[-1], cmap='gray')
plt.axis('off')
plt.show()

In [ ]:
# initialization
class_ids = dict()
confidences = dict()
boxes = dict()
conf_threshold = 0.5
nms_threshold = 0.4

# for each detetion from each output layer 
# get the confidence, class id, bounding box params
# and ignore weak detections (confidence < 0.5)
count = 0
for img in outs:
    class_ids[count] = []
    confidences[count] = []
    boxes[count] = []
    for out in img:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - w / 2
                y = center_y - h / 2
                class_ids[count].append(class_id)
                confidences[count].append(float(confidence))
                boxes[count].append([x, y, w, h])
    count += 1

In [ ]:
type(class_ids)

In [ ]:
count = 0
for image in images:
    indices = cv2.dnn.NMSBoxes(boxes[count], confidences[count], conf_threshold, nms_threshold)

    for i in indices:
        try:
            box = boxes[count][i]
        except:
            print(count, i)
            i = i[0]
            box = boxes[count][i]
        
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        draw_bounding_box(image, class_ids[count][i], confidences[count][i], round(x), round(y), round(x+w), round(y+h))
        img_resize = cv2.resize(image, (800, 600))
        #cv2.imshow("object detection", img_resize)
        #cv2.waitKey()
            
        cv2.imwrite("object-detection_{count}.jpg", image)
        count += 1
        cv2.destroyAllWindows()